https://machinelearningmastery.com/understanding-simple-recurrent-neural-networks-in-keras/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 1. Import thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [4]:
# 2. Đọc file csv và gắng index với giá Close
df = pd.read_csv('drive/MyDrive/data/Eximbank.csv')
df1=df.reset_index()['Close']

In [5]:
df1.shape

(1306,)

In [6]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [7]:
# 4. Chia train test
train_size = int(0.7 * len(df1))
test_size = len(df1) - train_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]


In [8]:
print(train_size)
print(test_size)

914
392


In [9]:
print(train_data.shape)
print(test_data.shape)


(914, 1)
(392, 1)


In [10]:
# 5. Hàm Create Dataset
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [11]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(ytest.shape)


(813, 100)
(813,)
(291, 100)
(291,)


In [13]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)


In [14]:
print(X_train.shape)
print(X_test.shape)

(813, 100, 1)
(291, 100, 1)


In [15]:
# 8. Define LSTM Model

model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [16]:
# 9. Fit mô hình với dữ liệu train
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
13/13 [==============================] - 11s 297ms/step - loss: 0.0366 - val_loss: 0.0095
Epoch 2/100
13/13 [==============================] - 2s 191ms/step - loss: 0.0086 - val_loss: 0.0027
Epoch 3/100
13/13 [==============================] - 2s 190ms/step - loss: 0.0047 - val_loss: 0.0028
Epoch 4/100
13/13 [==============================] - 4s 316ms/step - loss: 0.0039 - val_loss: 0.0038
Epoch 5/100
13/13 [==============================] - 3s 239ms/step - loss: 0.0037 - val_loss: 0.0025
Epoch 6/100
13/13 [==============================] - 2s 187ms/step - loss: 0.0036 - val_loss: 0.0027
Epoch 7/100
13/13 [==============================] - 2s 189ms/step - loss: 0.0035 - val_loss: 0.0025
Epoch 8/100
13/13 [==============================] - 2s 191ms/step - loss: 0.0034 - val_loss: 0.0027
Epoch 9/100
13/13 [==============================] - 3s 267ms/step - loss: 0.0033 - val_loss: 0.0022
Epoch 10/100
13/13 [==============================] - 4s 292ms/step - loss: 0.0032 - val_l

In [17]:
# 10. Dự báo dữ liệu test,
train_predict=model.predict(X_train)
y_pred=model.predict(test_data.reshape(test_data.shape[0],test_data.shape[1] , 1))


13/13 [==============================] - 1s 3ms/step


In [18]:
# 11. Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)


In [19]:
ytest=scaler.inverse_transform(test_data)
# 12. Đánh giá độ chính xác thuật toán bằng RMSE
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Testing RMSE:', test_rmse)

# Đánh giá độ chính xác thuật toán bằng MAPE
test_mape = np.mean(np.abs(ytest - y_pred)/np.abs(ytest))*100

print('Testing MAPE:', test_mape)

# Đánh giá độ chính xác thuật toán bằng MAE
test_mae = np.mean(abs(y_pred - ytest))
print('Testing MAE:', test_mae)

Testing RMSE: 5967.589875409371
Testing MAPE: 31.41537388888426
Testing MAE: 5772.394394629305


In [20]:
x_input=test_data[60:].reshape(1,-1)
x_input.shape

(1, 332)

In [21]:
# 13. Dự báo 30 ngày tiếp theo
x_input=test_data[60:].reshape(1,-1)
print(x_input.shape)

temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=332
i=0
while(i<90):

    if(len(temp_input)>332):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1


Streaming output truncated to the last 5000 lines.
 0.39012794 0.37477149 0.37696529 0.36380253 0.37477149 0.37915908
 0.39012794 0.39012794 0.39012794 0.37477149 0.37696529 0.38574036
 0.39670932 0.36819012 0.35502746 0.35063988 0.33967091 0.35283367
 0.35722125 0.35283367 0.35502746 0.38574036 0.37477149 0.36160874
 0.33308953 0.34625229 0.34625229 0.35502746 0.34844608 0.34625229
 0.36160874 0.38793415 0.40548449 0.47129797 0.45155394 0.50859235
 0.51517362 0.521755   0.52614259 0.52833638 0.52833638 0.56124317
 0.54808041 0.56563065 0.54808041 0.47349176 0.47349176 0.49762338
 0.49762338 0.49981718 0.51736742 0.583181   0.56124317 0.57440583
 0.60511882 0.58537479 0.58537479 0.58537479 0.583181   0.56124317
 0.57879341 0.57879341 0.57879341 0.50639855 0.47349176 0.49542959
 0.47349176 0.47910785 0.40921389 0.37038391 0.40403656 0.36779524
 0.3936819  0.38073857 0.33155393 0.34190859 0.3263766  0.34708592
 0.36002925 0.37556124 0.37814991 0.36779524 0.36779524 0.35744058
 0.35485192

In [22]:
# 14.Vẽ hình

train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))

test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,y_pred)

prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output[0:30]))

prediect_data_index = pd.RangeIndex(start=len(df1)+29, stop=len(df1)+59, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output[30:60]))

prediect_data_index = pd.RangeIndex(start=len(df1)+59, stop=len(df1)+89, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output[60:90]))
plt.legend(['Train','Test','Predict','Predict30days', 'Predict60days', 'Predict90days'])
plt.show()

In [23]:
# 14.Vẽ hình
# Dự đoán 90 ngày tiếp theo

prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output[0:30]))

prediect_data_index = pd.RangeIndex(start=len(df1)+29, stop=len(df1)+59, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output[30:60]))

prediect_data_index = pd.RangeIndex(start=len(df1)+59, stop=len(df1)+89, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output[60:90]))
plt.legend(['Predict30days', 'Predict60days', 'Predict90days'])
plt.show()

In [24]:
print(scaler.inverse_transform(lst_output))

[[20038.1335832 ]
 [20138.22776259]
 [20240.3527683 ]
 [20353.58745312]
 [20469.81887315]
 [20580.03725131]
 [20678.96635347]
 [20765.73352297]
 [20842.63092263]
 [20913.29999441]
 [20981.33152475]
 [21049.53344188]
 [21119.78575679]
 [21193.23167271]
 [21270.53661915]
 [21352.07130248]
 [21438.01071454]
 [21528.35715787]
 [21622.92182547]
 [21721.33285969]
 [21823.05837745]
 [21927.42258791]
 [22033.6898346 ]
 [22141.05999036]
 [22248.75365077]
 [22356.03055431]
 [22462.2126076 ]
 [22566.71381825]
 [22669.03914356]
 [22768.80175947]
 [22865.72651434]
 [22959.63726508]
 [23050.44766702]
 [23138.16232522]
 [23222.85376918]
 [23304.65439406]
 [23383.72192274]
 [23460.2578261 ]
 [23534.47048257]
 [23606.56251429]
 [23676.72503075]
 [23745.1422339 ]
 [23811.95918274]
 [23877.31518   ]
 [23941.28966276]
 [24003.96437063]
 [24065.3726902 ]
 [24125.52958789]
 [24184.42355107]
 [24242.03500829]
 [24298.32366538]
 [24353.24692563]
 [24406.75183099]
 [24458.78312089]
 [24509.30165241]
 [24558.26